# Stock API
Using [Yahoo Finance API](https://python-yahoofinance.readthedocs.io/en/latest/api.html) to get stock information.

# Import Libraries

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import ftplib
import io
import requests
import requests_html
from yahoo_fin.stock_info import get_data

In [12]:
amazon_weekly= get_data("amzn", start_date="12/04/2009", end_date="12/04/2019", index_as_date = True, interval="1d")
amazon_weekly

,open,high,low,close,adjclose,volume,ticker
2009-12-04,143.419998,143.449997,135.110001,137.580002,137.580002,14827400,AMZN
2009-12-07,138.000000,139.000000,133.839996,134.210007,134.210007,7837000,AMZN
2009-12-08,134.300003,136.080002,132.869995,134.110001,134.110001,8002800,AMZN
2009-12-09,134.600006,134.710007,129.820007,131.309998,131.309998,12632900,AMZN
2009-12-10,132.410004,136.190002,132.399994,135.380005,135.380005,11343600,AMZN
...,...,...,...,...,...,...,...
2019-11-26,1779.920044,1797.030029,1778.349976,1796.939941,1796.939941,3181200,AMZN
2019-11-27,1801.000000,1824.500000,1797.310059,1818.510010,1818.510010,3025600,AMZN
2019-11-29,1817.780029,1824.689941,1800.790039,1800.800049,1800.800049,1923400,AMZN
2019-12-02,1804.400024,1805.550049,1762.680054,1781.599976,1781.599976,3925600,AMZN
